#1.model

In [ ]:
import os
os.chdir('/content/drive/MyDrive/mini')

In [ ]:
!pip install subword-nmt

In [ ]:
import copy
from time import time
import os
import random
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, confusion_matrix, \
    precision_score, recall_score, auc
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import collections
import math
import copy
from subword_nmt.apply_bpe import BPE
import codecs

torch.manual_seed(2)
np.random.seed(3)

from model import BIN_Interaction_Flat1, BIN_Interaction_Flat2, BIN_Interaction_Flat3
from stream import BIN_Data_Encoder
from data import Cus_DataLoader

In [ ]:
def test(data_generator, model):
  torch.cuda.empty_cache()
  y_pred = []
  y_label = []
  model.eval()
  loss_accumulate = 0.0
  count = 0.0
  for i, (d, p, d_mask, p_mask, label) in enumerate(data_generator):
    score = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda()) # long tensor int64, .to(device)를 왜 사용안했지?

    m = torch.nn.Sigmoid()
    logits = torch.squeeze(m(score)) # 차원이 1인 축 제거
    loss_fct = torch.nn.BCELoss()

    label = label.float().cuda()

    loss = loss_fct(logits, label)

    loss_accumulate += loss.item()
    count += 1

    logits = logits.detach().cpu().numpy()

    label_ids = label.detach().to('cpu').numpy()
    y_label = y_label + label_ids.flatten().tolist()
    y_pred = y_pred + logits.flatten().tolist()

  loss = loss_accumulate / count

  fpr, tpr, thresholds = roc_curve(y_label, y_pred) # TPR: 양성을 잘 잡아내는 능력 값이 클수록 재현율이 높음, FPR: 음성을 얼마나 잘 음성으로 유지하는지 값이 작을수록 좋으며 음성을 양성으로 잘못 분류하는 비율이 낮음을 의미

  precision = tpr / (tpr + fpr+0.00001)

  f1 = 2 * precision * tpr / (tpr + precision + 0.00001)

  thred_optim = thresholds[2:][np.argmax(f1[2:])] # threshold는 y_pred에 대해 내림차순으로 뱉어내므로 너무 strict한 threshold 5개는 고려하지않음 그 이유는?

  print("optimal threshold: " + str(thred_optim))

  y_pred_s = [1 if i else 0 for i in (y_pred >= thred_optim)]

  auc_k = auc(fpr, tpr)
  print("AUROC:" + str(auc_k))
  print("AUPRC: " + str(average_precision_score(y_label, y_pred))) # 여러 threshold 기준 recall-precision auc

  cm1 = confusion_matrix(y_label, y_pred_s)
  print('Confusion Matrix : \n', cm1)
  print('Recall : ', recall_score(y_label, y_pred_s))
  print('Precision : ', precision_score(y_label, y_pred_s)) # 단일 threshold 기준(최적값)

  total1 = sum(sum(cm1))
  #####from confusion matrix calculate accuracy
  accuracy1 = (cm1[0, 0] + cm1[1, 1]) / total1
  print('Accuracy : ', accuracy1)

  sensitivity1 = cm1[0, 0] / (cm1[0, 0] + cm1[0, 1])
  print('Sensitivity : ', sensitivity1)

  specificity1 = cm1[1, 1] / (cm1[1, 0] + cm1[1, 1])
  print('Specificity : ', specificity1)

  outputs = np.asarray([1 if i else 0 for i in (np.asarray(y_pred) >= 0.5)])
  return roc_auc_score(y_label, y_pred), average_precision_score(y_label, y_pred), f1_score(y_label, outputs), y_pred, loss

def train(model,training_generator,epochs,batchsize,lr):
  torch.cuda.empty_cache()
  loss_history = []
  #model = model.cuda()

  opt = torch.optim.Adam(model.parameters(), lr=lr)
  print('--- Data Preparation ---')
  params = {'batch_size': batchsize, 'shuffle': True, 'drop_last': True}

  dataFolder = '/content/drive/MyDrive/mini'
  df_val = pd.read_csv(dataFolder + '/val.csv') #valid_new / test_new
  df_test = pd.read_csv(dataFolder + '/test.csv')

  validation_set = BIN_Data_Encoder(df_val.index.values, df_val.Label.values, df_val)
  validation_generator = DataLoader(validation_set, **params)

  testing_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, df_test)
  testing_generator = DataLoader(testing_set, **params)

  # early stopping
  max_auc = 0
  model_max = copy.deepcopy(model.to('cpu'))
  epo_count = 0

  with torch.no_grad(): # with torch.no_grad():
    model_max = model_max.cuda()
    auc, auprc, f1, logits, loss = test(testing_generator, model_max)
    print('Initial Testing AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: ' + str(f1) + ' , Test loss: ' + str(loss))
    model_max = model_max.cpu()
    torch.cuda.empty_cache()

  model=model.cuda()
  print('--- Go for Training ---')
  for epo in range(epochs):
    model.train()
    count=0.0
    loss_accumulate = 0.0
    for i,(d, p, d_mask, p_mask, label) in tqdm(enumerate(training_generator),desc="training"):
      score = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())

      label = label.float().cuda()

      loss_fct = torch.nn.BCELoss()
      m = torch.nn.Sigmoid()
      n = torch.squeeze(m(score))

      loss = loss_fct(n, label)

      loss_accumulate += loss.cpu().detach().numpy().item()
      count += 1

      opt.zero_grad()
      loss.backward()
      opt.step()

    loss = loss_accumulate / count
    loss_history.append(loss)
    torch.cuda.empty_cache()
    if ((epo+1) % 10 == 0):
      print('Training at Epoch ' + str(epo + 1) + ' with loss ' + str(loss))

    # every epoch test
    with torch.set_grad_enabled(False):
      auc, auprc, f1, logits, loss = test(validation_generator, model)
      if auc > max_auc:
        model_max = copy.deepcopy(model.to('cpu'))
        model=model.cuda()
        epo_count = 0
        max_auc = auc
        print('--------'+str(epo+1)+'model_max updated-------')
      else:
        epo_count += 1
      print('Validation at Epoch ' + str(epo + 1) + ' , AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: ' + str(f1) + ' , Validation loss: ' + str(loss))
      torch.cuda.empty_cache()
      if epo_count == 20 :
        break
  model.cpu()
  del model
  torch.cuda.empty_cache()
  print('--- Go for Testing ---')
  try:
    with torch.set_grad_enabled(False):
      model_max=model_max.cuda()
      auc, auprc, f1, logits, loss = test(testing_generator, model_max)
      print('Testing AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: ' + str(f1) + ' , Test loss: ' + str(loss))
      model_max=model_max.cpu()
      torch.cuda.empty_cache()
  except:
      print('testing failed')
  return model_max, loss_history

In [ ]:
def BIN_config_DBPE():
    config = {}
    config['batch_size'] = [4,32,64]
    config['input_dim_drug'] = 23532
    config['input_dim_target'] = 16693
    config['max_drug_seq'] = 50 #64
    config['max_protein_seq'] = 545 #576
    config['emb_size'] = 384
    config['dropout_rate'] = [0.1,0.15,0.2]

    #DenseNet
    # config['scale_down_ratio'] = 0.25
    # config['growth_rate'] = 20
    # config['transition_rate'] = 0.5
    # config['num_dense_blocks'] = 4
    config['kernal_dense_size'] = 3

    # Encoder
    config['intermediate_size'] = 1536
    config['num_attention_heads'] = 12
    config['attention_probs_dropout_prob'] = [0.1,0.15,0.2]
    config['hidden_dropout_prob'] = [0.1,0.15,0.2]
    config['flat_dim'] = 78192 #106764
    return config

In [ ]:
def main():
  batchsize = 4 # 16
  epochs =350 # 50
  #epochs_fine=50
  lr = 5e-5 #5e-5
  confidence=0.55
  ratio1=7
  ratio2=15
  config = BIN_config_DBPE()
  model = BIN_Interaction_Flat1(**config)
  model2 = BIN_Interaction_Flat2(**config)
  model3 = BIN_Interaction_Flat3(**config)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_cuda else "cpu")

  dataFolder = '/content/drive/MyDrive/mini'
  df_train = pd.read_csv(dataFolder + '/train.csv') #train_new
  df_unlabel = pd.read_csv(dataFolder + '/result.csv')

  params = {'batch_size': batchsize, 'shuffle': True, 'drop_last': True}
  params2 = {'batch_size': batchsize*16, 'shuffle': True, 'drop_last': True}
  training_set = BIN_Data_Encoder(df_train.index.values, df_train.Label.values, df_train)
  training_generator = DataLoader(training_set, **params)
  training_generator2 = DataLoader(training_set, **params2)

  #phase1
  print("-------phase1-------")
  # model_max, loss_history = train(model,training_generator,epochs,batchsize,lr)
  # torch.save(model_max, 'model_max.pt') # 임시추가
  # loss1= pd.Series(loss_history)
  # loss1.to_csv('loss1.csv',index=False)
  # torch.cuda.empty_cache()

  model_max=torch.load('model_max.pt')

  #phase2,3,4
  print("-------phase2--------")
  training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max, device, confidence, ratio1, batchsize)
  torch.cuda.empty_cache()
  model_max2, loss_history2 = train(model2,training_generator_with_unlabel,epochs*2,batchsize*8,lr)
  torch.save(model_max2, 'model_max2.pt') # 임시추가
  loss2= pd.Series(loss_history2)
  loss2.to_csv('loss2.csv',index=False)
  torch.cuda.empty_cache()
  # print("-------phase2 finetuning-------")
  # model_max2, loss_history2 = train(model_max2,training_generator2,epochs_fine*2,batchsize*4, lr)
  # torch.cuda.empty_cache()

  print("-------phase3--------")
  model_max2.batch_size=batchsize
  training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max2, device, confidence, ratio1, batchsize)
  torch.cuda.empty_cache()
  model_max3, loss_history3 = train(model2,training_generator_with_unlabel,epochs*2,batchsize*8,lr)
  torch.save(model_max3, 'model_max3.pt') # 임시추가
  loss3= pd.Series(loss_history3)
  loss3.to_csv('loss3.csv',index=False)
  torch.cuda.empty_cache()

  print("-------phase4--------")
  model_max3.batch_size=batchsize
  training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max3, device, confidence, ratio2, batchsize)
  torch.cuda.empty_cache()
  model_max4, loss_history4 = train(model3,training_generator_with_unlabel,epochs*2,batchsize*16,lr)
  torch.save(model_max4, 'model_max4.pt') # 임시추가
  loss4= pd.Series(loss_history4)
  loss4.to_csv('loss4.csv',index=False)
  torch.cuda.empty_cache()
  # print("-------phase3 finetuning-------")
  # model_max3, loss_history3 = train(model_max3,training_generator2,epochs_fine*3,batchsize*4,lr)
  # torch.cuda.empty_cache()

  print("-------phase5--------")
  model3 = BIN_Interaction_Flat3(**config)
  model_comparison, loss_history_comparison = train(model3, training_generator2, epochs*2, batchsize*16, lr)
  torch.save(model_comparison, 'model_comparison.pt') # 임시추가
  loss_comparison= pd.Series(loss_history_comparison)
  loss_comparison.to_csv('loss_comparison.csv',index=False)
  torch.cuda.empty_cache()

  print("-------training finished-------")

  return model_max, model_max2, model_max3, model_comparison, loss_history, loss_history2, loss_history3, loss_history_comparison

In [ ]:
s = time()
model_max, model_max2, model_max3, model_comparison, loss_history, loss_history2, loss_history3, loss_history_comparison = main()
e = time()
print((e - s)/60)

# torch.save(model_max, 'model_max.pt')
# torch.save(model_max2, 'model_max2.pt')
# torch.save(model_max3, 'model_max3.pt')
# torch.save(model_comparison, 'model_comparison.pt')
# loss= pd.Series(loss_history+loss_history2+loss_history3+loss_history_comparison)
# loss.to_csv('loss.csv',index=False)

from google.colab import runtime
runtime.unassign()

-------phase1-------


<ipython-input-6-6f3200f9355d>:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_max=torch.load('model_max.pt')


-------phase2--------
Labeling unlabeled data...
[1061750, 874214]
Labeling completed
--- Data Preparation ---
optimal threshold: 0.45665210485458374
AUROC:0.5219598397115395
AUPRC: 0.5251991124164315
Confusion Matrix : 
 [[   1 2714]
 [   0 2757]]
Recall :  1.0
Precision :  0.5039298117346006
Accuracy :  0.5040204678362573
Sensitivity :  0.00036832412523020257
Specificity :  1.0
Initial Testing AUROC: 0.5219598397115395 , AUPRC: 0.5251991124164315 , F1: 0.6604338070306657 , Test loss: 0.694316754215642
--- Go for Training ---


training: 28855it [1:35:34,  5.03it/s]


optimal threshold: 0.5733036994934082
AUROC:0.8747960356413689
AUPRC: 0.8816540151592005
Confusion Matrix : 
 [[1090  245]
 [ 275 1110]]
Recall :  0.8014440433212996
Precision :  0.8191881918819188
Accuracy :  0.8088235294117647
Sensitivity :  0.8164794007490637
Specificity :  0.8014440433212996
--------1model_max updated-------
Validation at Epoch 1 , AUROC: 0.8747960356413689 , AUPRC: 0.8816540151592005 , F1: 0.8071047957371226 , Validation loss: 0.5160341511754428


...

training: 28855it [1:35:37,  5.03it/s]


optimal threshold: 0.7687390446662903
AUROC:0.8874210346709239
AUPRC: 0.8833606877859261
Confusion Matrix : 
 [[1092  245]
 [ 244 1139]]
Recall :  0.8235719450469993
Precision :  0.8229768786127167
Accuracy :  0.8202205882352941
Sensitivity :  0.8167539267015707
Specificity :  0.8235719450469993
Validation at Epoch 13 , AUROC: 0.8874210346709239 , AUPRC: 0.8833606877859261 , F1: 0.8229868747782901 , Validation loss: 1.3897699359585256


training: 28855it [1:35:41,  5.03it/s]


optimal threshold: 0.09412975609302521
AUROC:0.8916326090236666
AUPRC: 0.8881421516470006
Confusion Matrix : 
 [[1070  267]
 [ 213 1170]]
Recall :  0.8459869848156182
Precision :  0.81419624217119
Accuracy :  0.8235294117647058
Sensitivity :  0.8002991772625281
Specificity :  0.8459869848156182
--------14model_max updated-------
Validation at Epoch 14 , AUROC: 0.8916326090236666 , AUPRC: 0.8881421516470006 , F1: 0.820830298616169 , Validation loss: 1.425496216731913


training: 5450it [18:04,  5.03it/s]

#2.testing

In [ ]:
batchsize = 8
epochs=1
epochs_fine=1
lr = 1e-4
confidence=0.55
ratio=3

config = BIN_config_DBPE()
model= BIN_Interaction_Flat1(**config)
model2 = BIN_Interaction_Flat3(**config)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

dataFolder = '/content/drive/MyDrive/mini'
df_train = pd.read_csv(dataFolder + '/train_new.csv')
df_unlabel = pd.read_csv(dataFolder + '/unlabel_new.csv')

params1={'batch_size': batchsize, 'shuffle': True, 'drop_last': True}

params2 = {'batch_size': batchsize*4, 'shuffle': True, 'drop_last': True}
training_set = BIN_Data_Encoder(df_train.index.values, df_train.Label.values, df_train)
training_generator = DataLoader(training_set, **params1)
training_generator2 = DataLoader(training_set, **params2)

training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model, device, confidence, ratio, batchsize)
model_max2, loss_history2 = train(model2,training_generator_with_unlabel,epochs*2,batchsize*4,lr)
loss= pd.Series(loss_history2)


Labeling unlabeled data...
[325, 8129]
Labeling completed
--- Data Preparation ---
optimal threshold: 0.46937626600265503
AUROC:0.47886936018473253
AUPRC: 0.4875089617843882
Confusion Matrix : 
 [[   2 1335]
 [   1 1350]]
Recall :  0.999259807549963
Precision :  0.5027932960893855
Accuracy :  0.5029761904761905
Sensitivity :  0.0014958863126402393
Specificity :  0.999259807549963
Initial Testing AUROC: 0.47886936018473253 , AUPRC: 0.4875089617843882 , F1: 0.004421518054532056 , Test loss: 0.6940490674404871
--- Go for Training ---
optimal threshold: 0.2340099811553955
AUROC:0.7411336059928406
AUPRC: 0.7961759616962455
Confusion Matrix : 
 [[268 366]
 [128 614]]
Recall :  0.8274932614555256
Precision :  0.6265306122448979
Accuracy :  0.6409883720930233
Sensitivity :  0.4227129337539432
Specificity :  0.8274932614555256
--------1model_max updated-------
Validation at Epoch 1 , AUROC: 0.7411336059928406 , AUPRC: 0.7961759616962455 , F1: 0.6597277822257807 , Validation loss: 0.656749489695

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
a=loss_history2[0]

In [ ]:
a.cpu()

tensor(0.4035, grad_fn=<ToCopyBackward0>)

In [ ]:
a.cpu().detach().numpy()

array(0.40350783, dtype=float32)

In [ ]:
config = BIN_config_DBPE()
model = BIN_Interaction_Flat1(**config)
#model_gpu=copy.deepcopy(model.cuda())
model=model.cpu()

In [ ]:
def testing(model):
  model=model.cuda()
  print(next(model.parameters()).device)
  del model

In [ ]:
testing(model)
print(next(model.parameters()).device)
#del model

cuda:0
cuda:0


In [ ]:
torch.cuda.empty_cache()

In [ ]:
ten=torch.tensor([1,2,3])

In [ ]:
ten=ten.cuda()

In [ ]:
ten.device

device(type='cuda', index=0)

In [ ]:
ten.cpu().device

device(type='cpu')

In [ ]:
ten.device

device(type='cuda', index=0)

In [ ]:
losss=pd.read_csv('loss1.csv')

In [ ]:
dataFolder = '/content/drive/MyDrive/mini'
df_train = pd.read_csv(dataFolder + '/train.csv') #train_new
df_unlabel = pd.read_csv(dataFolder + '/result.csv')
batchsize=4


In [ ]:
df_train=df_train[:500]

In [ ]:
df_unlabel=df_unlabel[:1000]

In [ ]:
params = {'batch_size': batchsize, 'shuffle': True, 'drop_last': True}
params2 = {'batch_size': batchsize*16, 'shuffle': True, 'drop_last': True}
training_set = BIN_Data_Encoder(df_train.index.values, df_train.Label.values, df_train)
training_generator = DataLoader(training_set, **params)

In [ ]:
model_max=torch.load('model_max.pt')
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
confidence=0.55
training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max, device, confidence, 7, 4)

<ipython-input-14-d89aa3e94c16>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_max=torch.load('model_max.pt')


Labeling unlabeled data...
[278, 211]
Labeling completed


In [ ]:
training_generator_with_unlabel.iter_count

6

In [ ]:
training_generator_with_unlabel.idx_unlabeled

1

In [ ]:
count=0
for _,_,_,_,_ in training_generator_with_unlabel:
  count+=1
print(count)

0


#3.analysis test

In [ ]:
import pandas as pd
import os

os.chdir('/content/drive/MyDrive/mini')

In [ ]:
!pip install subword-nmt

In [ ]:
import copy
from time import time
import os
import random
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, confusion_matrix, \
    precision_score, recall_score, auc
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import collections
import math
import copy
from subword_nmt.apply_bpe import BPE
import codecs

torch.manual_seed(2)
np.random.seed(3)
random.seed(4)

from model2 import BIN_Interaction_Flat1, BIN_Interaction_Flat2, BIN_Interaction_Flat3
from model import BIN_Interaction_Flat3 as BIN3
from stream import BIN_Data_Encoder
from data2 import Cus_DataLoader

In [ ]:
def test(data_generator, model):
  torch.cuda.empty_cache()
  y_pred = []
  y_label = []
  model.eval()
  loss_accumulate = 0.0
  count = 0.0
  for i, (d, p, d_mask, p_mask, label) in enumerate(data_generator):
    score = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda()) # long tensor int64, .to(device)를 왜 사용안했지?

    m = torch.nn.Sigmoid()
    logits = torch.squeeze(m(score)) # 차원이 1인 축 제거
    loss_fct = torch.nn.BCELoss()

    label = label.float().cuda()

    loss = loss_fct(logits, label)

    loss_accumulate += loss.item()
    count += 1

    logits = logits.detach().cpu().numpy()

    label_ids = label.detach().to('cpu').numpy()
    y_label = y_label + label_ids.flatten().tolist()
    y_pred = y_pred + logits.flatten().tolist()

  loss = loss_accumulate / count

  fpr, tpr, thresholds = roc_curve(y_label, y_pred) # TPR: 양성을 잘 잡아내는 능력 값이 클수록 재현율이 높음, FPR: 음성을 얼마나 잘 음성으로 유지하는지 값이 작을수록 좋으며 음성을 양성으로 잘못 분류하는 비율이 낮음을 의미

  precision = tpr / (tpr + fpr+0.00001)

  f1 = 2 * precision * tpr / (tpr + precision + 0.00001)

  thred_optim = thresholds[2:][np.argmax(f1[2:])] # threshold는 y_pred에 대해 내림차순으로 뱉어내므로 너무 strict한 threshold 5개는 고려하지않음 그 이유는?

  print("optimal threshold: " + str(thred_optim))

  y_pred_s = [1 if i else 0 for i in (y_pred >= thred_optim)]

  auc_k = auc(fpr, tpr)
  print("AUROC:" + str(auc_k))
  print("AUPRC: " + str(average_precision_score(y_label, y_pred))) # 여러 threshold 기준 recall-precision auc

  cm1 = confusion_matrix(y_label, y_pred_s)
  print('Confusion Matrix : \n', cm1)
  print('Recall : ', recall_score(y_label, y_pred_s))
  print('Precision : ', precision_score(y_label, y_pred_s)) # 단일 threshold 기준(최적값)

  total1 = sum(sum(cm1))
  #####from confusion matrix calculate accuracy
  accuracy1 = (cm1[0, 0] + cm1[1, 1]) / total1
  print('Accuracy : ', accuracy1)

  sensitivity1 = cm1[0, 0] / (cm1[0, 0] + cm1[0, 1])
  print('Sensitivity : ', sensitivity1)

  specificity1 = cm1[1, 1] / (cm1[1, 0] + cm1[1, 1])
  print('Specificity : ', specificity1)

  outputs = np.asarray([1 if i else 0 for i in (np.asarray(y_pred) >= 0.5)])
  return roc_auc_score(y_label, y_pred), average_precision_score(y_label, y_pred), f1_score(y_label, outputs), y_pred, loss

def train(model,training_generator,epochs,batchsize,lr):
  torch.cuda.empty_cache()
  loss_history = []
  loss_history2 = []
  #model = model.cuda()

  opt = torch.optim.Adam(model.parameters(), lr=lr)
  print('--- Data Preparation ---')
  params = {'batch_size': batchsize, 'shuffle': True, 'drop_last': True}

  dataFolder = '/content/drive/MyDrive/mini'
  df_val = pd.read_csv(dataFolder + '/valid_new.csv') #valid_new / test_new
  df_test = pd.read_csv(dataFolder + '/test_new.csv')

  validation_set = BIN_Data_Encoder(df_val.index.values, df_val.Label.values, df_val)
  validation_generator = DataLoader(validation_set, **params)

  testing_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, df_test)
  testing_generator = DataLoader(testing_set, **params)

  # early stopping
  max_auc = 0
  model_max = copy.deepcopy(model.to('cpu'))
  epo_count = 0

  with torch.no_grad(): # with torch.no_grad():
    model_max = model_max.cuda()
    auc, auprc, f1, logits, loss = test(testing_generator, model_max)
    print('Initial Testing AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: ' + str(f1) + ' , Test loss: ' + str(loss))
    model_max = model_max.cpu()
    torch.cuda.empty_cache()

  model=model.cuda()
  print('--- Go for Training ---')
  for epo in range(epochs):
    model.train()
    count=0.0
    loss_accumulate = 0.0
    for i,(d, p, d_mask, p_mask, label) in tqdm(enumerate(training_generator),desc="training"):
      score = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())

      label = label.float().cuda()

      loss_fct = torch.nn.BCELoss()
      m = torch.nn.Sigmoid()
      n = torch.squeeze(m(score))

      loss = loss_fct(n, label)

      loss_accumulate += loss.cpu().detach().numpy().item()
      count += 1

      opt.zero_grad()
      loss.backward()
      opt.step()

    loss = loss_accumulate / count
    loss_history.append(loss)
    torch.cuda.empty_cache()
    if ((epo+1) % 10 == 0):
      print('Training at Epoch ' + str(epo + 1) + ' with loss ' + str(loss))

    # every epoch test
    with torch.set_grad_enabled(False):
      auc, auprc, f1, logits, loss = test(validation_generator, model)
      loss_history2.append(loss)
      if auc > max_auc:
        model_max = copy.deepcopy(model.to('cpu'))
        model=model.cuda()
        epo_count = 0
        max_auc = auc
        print('--------'+str(epo+1)+'model_max updated-------')
      else:
        epo_count += 1
      print('Validation at Epoch ' + str(epo + 1) + ' , AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: ' + str(f1) + ' , Validation loss: ' + str(loss))
      torch.cuda.empty_cache()
      if epo_count == 30 :
        break
  model.cpu()
  del model
  torch.cuda.empty_cache()
  print('--- Go for Testing ---')
  try:
    with torch.set_grad_enabled(False):
      model_max=model_max.cuda()
      auc, auprc, f1, logits, loss = test(testing_generator, model_max)
      print('Testing AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: ' + str(f1) + ' , Test loss: ' + str(loss))
      model_max=model_max.cpu()
      torch.cuda.empty_cache()
  except:
      print('testing failed')
  return model_max, loss_history, loss_history2

In [ ]:
def BIN_config_DBPE():
    config = {}
    config['batch_size'] = [4,32,64]
    config['input_dim_drug'] = 23532
    config['input_dim_target'] = 16693
    config['max_drug_seq'] = 50 #64
    config['max_protein_seq'] = 545 #576
    config['emb_size'] = 384
    config['dropout_rate'] = [0.1,0.15,0.1]

    #DenseNet
    # config['scale_down_ratio'] = 0.25
    # config['growth_rate'] = 20
    # config['transition_rate'] = 0.5
    # config['num_dense_blocks'] = 4
    config['kernal_dense_size'] = 3

    # Encoder
    config['intermediate_size'] = 1536
    config['num_attention_heads'] = 12
    config['attention_probs_dropout_prob'] = [0.1,0.15,0.1]
    config['hidden_dropout_prob'] = [0.1,0.15,0.1]
    config['flat_dim'] = 78192 #106764
    return config

In [ ]:
def main():
  batchsize = 4 # 16
  epochs =120 # 50
  lr1 = 5e-5 #5e-5
  lr = 1e-4
  confidence=0.55
  ratio1=7
  ratio2=15
  config = BIN_config_DBPE()
  model = BIN_Interaction_Flat1(**config)
  model2 = BIN_Interaction_Flat2(**config)
  model3 = BIN_Interaction_Flat3(**config)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_cuda else "cpu")

  dataFolder = '/content/drive/MyDrive/mini'
  df_train = pd.read_csv(dataFolder + '/train_new.csv') #train_new
  df_unlabel = pd.read_csv(dataFolder + '/unlabel_new.csv')

  params = {'batch_size': batchsize, 'shuffle': True, 'drop_last': True}
  params2 = {'batch_size': batchsize*16, 'shuffle': True, 'drop_last': True}
  training_set = BIN_Data_Encoder(df_train.index.values, df_train.Label.values, df_train)
  training_generator = DataLoader(training_set, **params)
  training_generator2 = DataLoader(training_set, **params2)

  #phase1
  print("-------phase1-------")
  # model_max, loss_history, loss_historyv = train(model,training_generator,epochs,batchsize,lr1)
  # torch.save(model_max, './new_test1/model_max.pt') # 임시추가
  # loss1= pd.Series(loss_history)
  # loss1.to_csv('./new_test1/loss1.csv',index=False)
  # loss1v= pd.Series(loss_historyv)
  # loss1v.to_csv('./new_test1/loss1v.csv',index=False)
  # torch.cuda.empty_cache()

  # model_max=torch.load('./new_test_param/model_max.pt')

  #phase2,3,4,5,6
  #print("-------phase2--------")
  # training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max, device, confidence, ratio1, batchsize)
  # torch.cuda.empty_cache()
  # model_max2, loss_history2, loss_history2v = train(model2,training_generator_with_unlabel,epochs*2,batchsize*8,lr)
  # torch.save(model_max2, './new_test_param/model_max2.pt') # 임시추가
  # loss2= pd.Series(loss_history2)
  # loss2.to_csv('./new_test_param/loss2.csv',index=False)
  # loss2v= pd.Series(loss_history2v)
  # loss2v.to_csv('./new_test_param/loss2v.csv',index=False)
  # torch.cuda.empty_cache()

  # model_max2=torch.load('./new_test_param/model_max2.pt')

  # print("-------phase3--------")
  # model_max2.batch_size=batchsize
  # training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max2, device, confidence, ratio1, batchsize)
  # torch.cuda.empty_cache()
  # model_max3, loss_history3, loss_history3v = train(model2,training_generator_with_unlabel,epochs*2,batchsize*8,lr)
  # torch.save(model_max3, './new_test_param/model_max3.pt') # 임시추가
  # loss3= pd.Series(loss_history3)
  # loss3.to_csv('./new_test_param/loss3.csv',index=False)
  # loss3v= pd.Series(loss_history3v)
  # loss3v.to_csv('./new_test_param/loss3v.csv',index=False)
  # torch.cuda.empty_cache()

  # # model_max3=torch.load('./new_test1/model_max3.pt')

  # print("-------phase4--------")
  # model_max3.batch_size=batchsize
  # training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max3, device, confidence, ratio2, batchsize)
  # torch.cuda.empty_cache()
  # model_max4, loss_history4, loss_history4v = train(model3,training_generator_with_unlabel,epochs*2,batchsize*16,lr)
  # torch.save(model_max4, './new_test_param/model_max4.pt') # 임시추가
  # loss4= pd.Series(loss_history4)
  # loss4.to_csv('./new_test_param/loss4.csv',index=False)
  # loss4v= pd.Series(loss_history4v)
  # loss4v.to_csv('./new_test_param/loss4v.csv',index=False)
  # torch.cuda.empty_cache()

  # print("-------phase5--------") # 모델 성능만 봄
  # model3 = BIN_Interaction_Flat3(**config)
  # model3.batch_size=batchsize
  # model_comparison, loss_history_comparison, loss_history_comparisonv= train(model3, training_generator, epochs*2, batchsize, lr1)
  # torch.save(model_comparison, './new_test_noise/model_comparison.pt') # 임시추가
  # loss_comparison= pd.Series(loss_history_comparison)
  # loss_comparison.to_csv('./new_test_noise/loss_comparison.csv',index=False)
  # loss_comparisonv= pd.Series(loss_history_comparisonv)
  # loss_comparisonv.to_csv('./new_test_noise/loss_comparisonv.csv',index=False)
  # torch.cuda.empty_cache()

  # print("-------phase6--------") # iteration 구조 성능
  # model3 = BIN_Interaction_Flat3(**config)
  # training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max, device, confidence, ratio2, batchsize)
  # torch.cuda.empty_cache()
  # model_comparison2, loss_history_comparison2, loss_history_comparison2v = train(model3, training_generator_with_unlabel, epochs*2, batchsize*16, lr)
  # torch.save(model_comparison2, './new_test_param/model_comparison2.pt') # 임시추가
  # loss_comparison2= pd.Series(loss_history_comparison2)
  # loss_comparison2.to_csv('./new_test_param/loss_comparison2.csv',index=False)
  # loss_comparison2v= pd.Series(loss_history_comparison2v)
  # loss_comparison2v.to_csv('./new_test_param/loss_comparison2v.csv',index=False)
  # torch.cuda.empty_cache()

  # print("-------error fix3--------") # iteration 구조 성능
  # model3 = BIN_Interaction_Flat3(**config)
  # training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max, device, confidence, ratio2, batchsize)
  # torch.cuda.empty_cache()
  # model_comparison2, loss_history_comparison2, loss_history_comparison2v = train(model3, training_generator_with_unlabel, epochs*2, batchsize*16, lr)
  # torch.save(model_comparison2, './new_test_noise/model_comparison3.pt') # 임시추가
  # loss_comparison2= pd.Series(loss_history_comparison2)
  # loss_comparison2.to_csv('./new_test_noise/loss_comparison3.csv',index=False)
  # loss_comparison2v= pd.Series(loss_history_comparison2v)
  # loss_comparison2v.to_csv('./new_test_noise/loss_comparison3v.csv',index=False)
  # torch.cuda.empty_cache()

  print("-------training finished-------")

  return

In [ ]:
s = time()
main()
e = time()
print((e - s)/60)

from google.colab import runtime
runtime.unassign()

-------phase1-------


<ipython-input-6-fa6f748b3131>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_max=torch.load('./new_test_param/model_max.pt')


-------error fix3--------
Labeling unlabeled data...
[7618, 5062]
Labeling completed
--- Data Preparation ---
optimal threshold: 0.6414676904678345
AUROC:0.49153158390079166
AUPRC: 0.494353786784544
Confusion Matrix : 
 [[   0 1338]
 [   0 1350]]
Recall :  1.0
Precision :  0.5022321428571429
Accuracy :  0.5022321428571429
Sensitivity :  0.0
Specificity :  1.0
Initial Testing AUROC: 0.49153158390079166 , AUPRC: 0.494353786784544 , F1: 0.6686478454680534 , Test loss: 0.9071681939420246
--- Go for Training ---


training: 2500it [09:46,  4.26it/s]


optimal threshold: 0.7277835011482239
AUROC:0.838610063565933
AUPRC: 0.8447312428117073
Confusion Matrix : 
 [[503 115]
 [164 562]]
Recall :  0.7741046831955923
Precision :  0.8301329394387001
Accuracy :  0.7924107142857143
Sensitivity :  0.813915857605178
Specificity :  0.7741046831955923
--------1model_max updated-------
Validation at Epoch 1 , AUROC: 0.838610063565933 , AUPRC: 0.8447312428117073 , F1: 0.7938288920056101 , Validation loss: 1.9365072619347345


...

Training at Epoch 100 with loss 0.0016498518402317131
optimal threshold: 0.3024058938026428
AUROC:0.8636465777318963
AUPRC: 0.8853540472099141
Confusion Matrix : 
 [[517 109]
 [149 569]]
Recall :  0.7924791086350975
Precision :  0.8392330383480826
Accuracy :  0.8080357142857143
Sensitivity :  0.8258785942492013
Specificity :  0.7924791086350975
Validation at Epoch 100 , AUROC: 0.8636465777318963 , AUPRC: 0.8853540472099141 , F1: 0.8118240807498197 , Validation loss: 1.7196758361089797


training: 2500it [09:46,  4.26it/s]


optimal threshold: 0.901019811630249
AUROC:0.8585202991452991
AUPRC: 0.8742732460612889
Confusion Matrix : 
 [[530  94]
 [174 546]]
Recall :  0.7583333333333333
Precision :  0.853125
Accuracy :  0.8005952380952381
Sensitivity :  0.8493589743589743
Specificity :  0.7583333333333333
Validation at Epoch 101 , AUROC: 0.8585202991452991 , AUPRC: 0.8742732460612889 , F1: 0.7997128499641063 , Validation loss: 2.0018480051131475


training: 2500it [09:46,  4.26it/s]


optimal threshold: 0.5124270915985107
AUROC:0.8620136622779213
AUPRC: 0.883298723071994
Confusion Matrix : 
 [[506 121]
 [144 573]]
Recall :  0.799163179916318
Precision :  0.8256484149855908
Accuracy :  0.8028273809523809
Sensitivity :  0.8070175438596491
Specificity :  0.799163179916318
Validation at Epoch 102 , AUROC: 0.8620136622779213 , AUPRC: 0.883298723071994 , F1: 0.81218993621545 , Validation loss: 1.9200719992319744
--- Go for Testing ---
optimal threshold: 0.05164109915494919
AUROC:0.8555055026515368
AUPRC: 0.8714305086061892
Confusion Matrix : 
 [[1003  336]
 [ 263 1086]]
Recall :  0.8050407709414381
Precision :  0.7637130801687764
Accuracy :  0.7771577380952381
Sensitivity :  0.7490664675130695
Specificity :  0.8050407709414381
Testing AUROC: 0.8555055026515368 , AUPRC: 0.8714305086061892 , F1: 0.7790262172284644 , Test loss: 1.7502103391147794
-------training finished-------
1002.1450723846754


In [ ]:
def BIN_config_DBPE():
    config = {}
    config['batch_size'] = [4,64,128]
    config['input_dim_drug'] = 23532
    config['input_dim_target'] = 16693
    config['max_drug_seq'] = 50 #64
    config['max_protein_seq'] = 545 #576
    config['emb_size'] = 384
    config['dropout_rate'] = [0.1,0.15,0.2]

    #DenseNet
    # config['scale_down_ratio'] = 0.25
    # config['growth_rate'] = 20
    # config['transition_rate'] = 0.5
    # config['num_dense_blocks'] = 4
    config['kernal_dense_size'] = 3

    # Encoder
    config['intermediate_size'] = 1536
    config['num_attention_heads'] = 12
    config['attention_probs_dropout_prob'] = [0.1,0.15,0.2]
    config['hidden_dropout_prob'] = [0.1,0.15,0.2]
    config['flat_dim'] = 78192 #106764
    return config

In [ ]:
def main():
  batchsize = 4 # 16
  epochs =120 # 50
  lr1 = 5e-5 #5e-5
  lr = 1e-4
  confidence=0.55
  ratio1=15
  ratio2=31
  config = BIN_config_DBPE()
  model = BIN_Interaction_Flat1(**config)
  model2 = BIN_Interaction_Flat2(**config)
  model3 = BIN_Interaction_Flat3(**config)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_cuda else "cpu")

  dataFolder = '/content/drive/MyDrive/mini'
  df_train = pd.read_csv(dataFolder + '/train_new.csv') #train_new
  df_unlabel = pd.read_csv(dataFolder + '/unlabel_new.csv')

  params = {'batch_size': batchsize, 'shuffle': True, 'drop_last': True}
  params2 = {'batch_size': batchsize*16, 'shuffle': True, 'drop_last': True}
  training_set = BIN_Data_Encoder(df_train.index.values, df_train.Label.values, df_train)
  training_generator = DataLoader(training_set, **params)
  training_generator2 = DataLoader(training_set, **params2)

  #phase1
  print("-------phase1-------")
  # model_max, loss_history, loss_historyv = train(model,training_generator,epochs,batchsize,lr1)
  # torch.save(model_max, './new_test1/model_max.pt') # 임시추가
  # loss1= pd.Series(loss_history)
  # loss1.to_csv('./new_test1/loss1.csv',index=False)
  # loss1v= pd.Series(loss_historyv)
  # loss1v.to_csv('./new_test1/loss1v.csv',index=False)
  # torch.cuda.empty_cache()

  model_max=torch.load('./new_test_batch/model_max.pt')

  #phase2,3,4,5,6
  print("-------phase2--------")
  # training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max, device, confidence, ratio1, batchsize)
  # torch.cuda.empty_cache()
  # model_max2, loss_history2, loss_history2v = train(model2,training_generator_with_unlabel,epochs*2,batchsize*16,lr)
  # torch.save(model_max2, './new_test_batch/model_max2.pt') # 임시추가
  # loss2= pd.Series(loss_history2)
  # loss2.to_csv('./new_test_batch/loss2.csv',index=False)
  # loss2v= pd.Series(loss_history2v)
  # loss2v.to_csv('./new_test_batch/loss2v.csv',index=False)
  # torch.cuda.empty_cache()

  model_max2=torch.load('./new_test_batch/model_max2.pt')


  print("-------phase3--------")
  model_max2.batch_size=batchsize
  training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max2, device, confidence, ratio1, batchsize)
  torch.cuda.empty_cache()
  model_max3, loss_history3, loss_history3v = train(model2,training_generator_with_unlabel,epochs*2,batchsize*16,lr)
  torch.save(model_max3, './new_test_batch/model_max3.pt') # 임시추가
  loss3= pd.Series(loss_history3)
  loss3.to_csv('./new_test_batch/loss3.csv',index=False)
  loss3v= pd.Series(loss_history3v)
  loss3v.to_csv('./new_test_batch/loss3v.csv',index=False)
  torch.cuda.empty_cache()

  # model_max3=torch.load('./new_test1/model_max3.pt')

  print("-------phase4--------")
  model_max3.batch_size=batchsize
  training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max3, device, confidence, ratio2, batchsize)
  torch.cuda.empty_cache()
  model_max4, loss_history4, loss_history4v = train(model3,training_generator_with_unlabel,epochs*2,batchsize*32,lr)
  torch.save(model_max4, './new_test_batch/model_max4.pt') # 임시추가
  loss4= pd.Series(loss_history4)
  loss4.to_csv('./new_test_batch/loss4.csv',index=False)
  loss4v= pd.Series(loss_history4v)
  loss4v.to_csv('./new_test_batch/loss4v.csv',index=False)
  torch.cuda.empty_cache()

  print("-------phase5--------") # 모델 성능만 봄
  model3 = BIN_Interaction_Flat3(**config)
  model3.batch_size=batchsize
  model_comparison, loss_history_comparison, loss_history_comparisonv= train(model3, training_generator, epochs*2, batchsize, lr1)
  torch.save(model_comparison, './new_test_batch/model_comparison.pt') # 임시추가
  loss_comparison= pd.Series(loss_history_comparison)
  loss_comparison.to_csv('./new_test_batch/loss_comparison.csv',index=False)
  loss_comparisonv= pd.Series(loss_history_comparisonv)
  loss_comparisonv.to_csv('./new_test_batch/loss_comparisonv.csv',index=False)
  torch.cuda.empty_cache()

  print("-------phase6--------") # iteration 구조 성능
  model3 = BIN_Interaction_Flat3(**config)
  training_generator_with_unlabel=Cus_DataLoader(training_generator, df_unlabel, model_max, device, confidence, ratio2, batchsize)
  torch.cuda.empty_cache()
  model_comparison2, loss_history_comparison2, loss_history_comparison2v = train(model3, training_generator_with_unlabel, epochs*2, batchsize*32, lr)
  torch.save(model_comparison2, './new_test_batch/model_comparison2.pt') # 임시추가
  loss_comparison2= pd.Series(loss_history_comparison2)
  loss_comparison2.to_csv('./new_test_batch/loss_comparison2.csv',index=False)
  loss_comparison2v= pd.Series(loss_history_comparison2v)
  loss_comparison2v.to_csv('./new_test_batch/loss_comparison2v.csv',index=False)
  torch.cuda.empty_cache()

  print("-------training finished-------")

  return

In [ ]:
s = time()
main()
e = time()
print((e - s)/60)

from google.colab import runtime
runtime.unassign()

-------phase1-------


<ipython-input-6-7f78c7e868bc>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_max=torch.load('./new_test_batch/model_max.pt')


-------phase2--------


<ipython-input-6-7f78c7e868bc>:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_max2=torch.load('./new_test_batch/model_max2.pt')


-------phase3--------
Labeling unlabeled data...
[6917, 6411]
Labeling completed
--- Data Preparation ---
optimal threshold: 0.48674318194389343
AUROC:0.48835622467525924
AUPRC: 0.4954932060887357
Confusion Matrix : 
 [[   5 1332]
 [   1 1350]]
Recall :  0.999259807549963
Precision :  0.5033557046979866
Accuracy :  0.5040922619047619
Sensitivity :  0.0037397157816005983
Specificity :  0.999259807549963
Initial Testing AUROC: 0.48835622467525924 , AUPRC: 0.4954932060887357 , F1: 0.6668329177057356 , Test loss: 0.7121045163699559
--- Go for Training ---


...

optimal threshold: 0.04449218511581421
AUROC:0.8357200142577081
AUPRC: 0.8274074396565458
Confusion Matrix : 
 [[480 140]
 [149 575]]
Recall :  0.7941988950276243
Precision :  0.8041958041958042
Accuracy :  0.7849702380952381
Sensitivity :  0.7741935483870968
Specificity :  0.7941988950276243
Validation at Epoch 31 , AUROC: 0.8357200142577081 , AUPRC: 0.8274074396565458 , F1: 0.7859195402298851 , Validation loss: 1.7309840406690324


training: 2500it [09:47,  4.25it/s]


optimal threshold: 0.031170733273029327
AUROC:0.8347310561619653
AUPRC: 0.83632546458951
Confusion Matrix : 
 [[488 135]
 [148 573]]
Recall :  0.7947295423023578
Precision :  0.809322033898305
Accuracy :  0.7894345238095238
Sensitivity :  0.78330658105939
Specificity :  0.7947295423023578
Validation at Epoch 32 , AUROC: 0.8347310561619653 , AUPRC: 0.83632546458951 , F1: 0.7865497076023392 , Validation loss: 1.8171548077038355
--- Go for Testing ---
optimal threshold: 0.9424504041671753
AUROC:0.8281884329566674
AUPRC: 0.8360527536754667
Confusion Matrix : 
 [[1042  295]
 [ 325 1026]]
Recall :  0.7594374537379719
Precision :  0.7766843300529902
Accuracy :  0.7693452380952381
Sensitivity :  0.7793567688855647
Specificity :  0.7594374537379719
Testing AUROC: 0.8281884329566674 , AUPRC: 0.8360527536754667 , F1: 0.7435736677115987 , Test loss: 1.0371413429578145
-------phase4--------
Labeling unlabeled data...
[4637, 8363]
Labeling completed
--- Data Preparation ---
optimal threshold: 0.2403

training: 0it [00:00, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.99 GiB. GPU 0 has a total capacity of 39.56 GiB of which 368.88 MiB is free. Process 22696 has 39.19 GiB memory in use. Of the allocated memory 37.70 GiB is allocated by PyTorch, and 1011.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

#4.analysis test2

In [1]:
import pandas as pd
import os

os.chdir('/content/drive/MyDrive/mini')

In [2]:
!pip install subword-nmt

In [3]:
import copy
from time import time
import os
import random
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, confusion_matrix, \
    precision_score, recall_score, auc
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import collections
import math
import copy
from subword_nmt.apply_bpe import BPE
import codecs

torch.manual_seed(2)
np.random.seed(3)
random.seed(4)

from model2 import BIN_Interaction_Flat1, BIN_Interaction_Flat2, BIN_Interaction_Flat3
from model import BIN_Interaction_Flat3 as BIN3
from stream import BIN_Data_Encoder
from data2 import Cus_DataLoader

In [4]:
def test(data_generator, model):
  torch.cuda.empty_cache()
  y_pred = []
  y_label = []
  model.eval()
  loss_accumulate = 0.0
  count = 0.0
  for i, (d, p, d_mask, p_mask, label) in enumerate(data_generator):
    score = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda()) # long tensor int64, .to(device)를 왜 사용안했지?

    m = torch.nn.Sigmoid()
    logits = torch.squeeze(m(score)) # 차원이 1인 축 제거
    loss_fct = torch.nn.BCELoss()

    label = label.float().cuda()

    loss = loss_fct(logits, label)

    loss_accumulate += loss.item()
    count += 1

    logits = logits.detach().cpu().numpy()

    label_ids = label.detach().to('cpu').numpy()
    y_label = y_label + label_ids.flatten().tolist()
    y_pred = y_pred + logits.flatten().tolist()

  loss = loss_accumulate / count

  fpr, tpr, thresholds = roc_curve(y_label, y_pred) # TPR: 양성을 잘 잡아내는 능력 값이 클수록 재현율이 높음, FPR: 음성을 얼마나 잘 음성으로 유지하는지 값이 작을수록 좋으며 음성을 양성으로 잘못 분류하는 비율이 낮음을 의미

  precision = tpr / (tpr + fpr+0.00001)

  f1 = 2 * precision * tpr / (tpr + precision + 0.00001)

  thred_optim = thresholds[2:][np.argmax(f1[2:])] # threshold는 y_pred에 대해 내림차순으로 뱉어내므로 너무 strict한 threshold 5개는 고려하지않음 그 이유는?

  print("optimal threshold: " + str(thred_optim))

  y_pred_s = [1 if i else 0 for i in (y_pred >= thred_optim)]

  auc_k = auc(fpr, tpr)
  print("AUROC:" + str(auc_k))
  print("AUPRC: " + str(average_precision_score(y_label, y_pred))) # 여러 threshold 기준 recall-precision auc

  cm1 = confusion_matrix(y_label, y_pred_s)
  print('Confusion Matrix : \n', cm1)
  print('Recall : ', recall_score(y_label, y_pred_s))
  print('Precision : ', precision_score(y_label, y_pred_s)) # 단일 threshold 기준(최적값)

  total1 = sum(sum(cm1))
  #####from confusion matrix calculate accuracy
  accuracy1 = (cm1[0, 0] + cm1[1, 1]) / total1
  print('Accuracy : ', accuracy1)

  sensitivity1 = cm1[0, 0] / (cm1[0, 0] + cm1[0, 1]) # sensitivity랑 specificity 잘못선택됨
  print('Sensitivity : ', sensitivity1)

  specificity1 = cm1[1, 1] / (cm1[1, 0] + cm1[1, 1])
  print('Specificity : ', specificity1)

  outputs = np.asarray([1 if i else 0 for i in (np.asarray(y_pred) >= 0.5)])
  return roc_auc_score(y_label, y_pred), average_precision_score(y_label, y_pred), f1_score(y_label, outputs), y_pred, loss

def train(model,training_generator,epochs,batchsize,lr):
  torch.cuda.empty_cache()
  loss_history = []
  loss_history2 = []
  #model = model.cuda()

  opt = torch.optim.Adam(model.parameters(), lr=lr)
  print('--- Data Preparation ---')
  params = {'batch_size': batchsize, 'shuffle': True, 'drop_last': True}

  dataFolder = '/content/drive/MyDrive/mini'
  df_val = pd.read_csv(dataFolder + '/valid_new.csv') #valid_new / test_new
  df_test = pd.read_csv(dataFolder + '/test_new.csv')

  validation_set = BIN_Data_Encoder(df_val.index.values, df_val.Label.values, df_val)
  validation_generator = DataLoader(validation_set, **params)

  testing_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, df_test)
  testing_generator = DataLoader(testing_set, **params)

  # early stopping
  max_auc = 0
  model_max = copy.deepcopy(model.to('cpu'))
  epo_count = 0

  with torch.no_grad(): # with torch.no_grad():
    model_max = model_max.cuda()
    auc, auprc, f1, logits, loss = test(testing_generator, model_max)
    print('Initial Testing AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: ' + str(f1) + ' , Test loss: ' + str(loss))
    model_max = model_max.cpu()
    torch.cuda.empty_cache()

  model=model.cuda()
  print('--- Go for Training ---')
  for epo in range(epochs):
    model.train()
    count=0.0
    loss_accumulate = 0.0
    for i,(d, p, d_mask, p_mask, label) in tqdm(enumerate(training_generator),desc="training"):
      score = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())

      label = label.float().cuda()

      loss_fct = torch.nn.BCELoss()
      m = torch.nn.Sigmoid()
      n = torch.squeeze(m(score))

      loss = loss_fct(n, label)

      loss_accumulate += loss.cpu().detach().numpy().item()
      count += 1

      opt.zero_grad()
      loss.backward()
      opt.step()

    loss = loss_accumulate / count
    loss_history.append(loss)
    torch.cuda.empty_cache()
    if ((epo+1) % 10 == 0):
      print('Training at Epoch ' + str(epo + 1) + ' with loss ' + str(loss))

    # every epoch test
    with torch.set_grad_enabled(False):
      auc, auprc, f1, logits, loss = test(validation_generator, model)
      loss_history2.append(loss)
      if auc > max_auc:
        model_max = copy.deepcopy(model.to('cpu'))
        model=model.cuda()
        epo_count = 0
        max_auc = auc
        print('--------'+str(epo+1)+'model_max updated-------')
      else:
        epo_count += 1
      print('Validation at Epoch ' + str(epo + 1) + ' , AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: ' + str(f1) + ' , Validation loss: ' + str(loss))
      torch.cuda.empty_cache()
      if epo_count == 30 :
        break
  model.cpu()
  del model
  torch.cuda.empty_cache()
  print('--- Go for Testing ---')
  try:
    with torch.set_grad_enabled(False):
      model_max=model_max.cuda()
      auc, auprc, f1, logits, loss = test(testing_generator, model_max)
      print('Testing AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: ' + str(f1) + ' , Test loss: ' + str(loss))
      model_max=model_max.cpu()
      torch.cuda.empty_cache()
  except:
      print('testing failed')
  return model_max, loss_history, loss_history2

In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/mini/unlabel_new.csv')

In [ ]:
batchsize=64

params = {'batch_size': batchsize, 'shuffle': True, 'drop_last': True}

df_test = pd.read_csv('/content/drive/MyDrive/mini/unlabel_new.csv')

testing_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, df_test)
testing_generator = DataLoader(testing_set, **params)

model_test=torch.load('./new_test1/model_comparison3.pt')
model_test=model_test.cuda()

<ipython-input-6-1b97d3ca7fe2>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_test=torch.load('./new_test1/model_comparison3.pt')


In [ ]:
test(testing_generator, model_test)

optimal threshold: 0.19230510294437408
AUROC:0.843440304355723
AUPRC: 0.844725388684997
Confusion Matrix : 
 [[5243 1479]
 [1390 5264]]
Recall :  0.7911030958821761
Precision :  0.7806614266646893
Accuracy :  0.7855113636363636
Sensitivity :  0.7799761975602499
Specificity :  0.7911030958821761


(0.843440304355723,
 0.844725388684997,
 0.7832093517534537,
 [0.9998935461044312,
  0.99972003698349,
  0.9992755055427551,
  0.9984544515609741,
  0.9992275238037109,
  0.003459375351667404,
  0.003927960060536861,
  0.0008045454742386937,
  0.9995624423027039,
  0.09005719423294067,
  0.00017833488527685404,
  0.9983518123626709,
  0.9548445343971252,
  0.9997830986976624,
  0.9988930821418762,
  0.9999141693115234,
  0.9969792366027832,
  0.9998621940612793,
  0.9993482232093811,
  0.9983868598937988,
  0.9954195022583008,
  0.8648730516433716,
  0.0017506207805126905,
  0.00034066420630551875,
  0.0005016125505790114,
  0.999647855758667,
  0.67743319272995,
  0.0004770485684275627,
  0.9003475308418274,
  0.9997898936271667,
  0.9956900477409363,
  0.9997758269309998,
  0.003089766949415207,
  0.0002175715781049803,
  0.9958686828613281,
  0.9985104203224182,
  0.07901182770729065,
  0.9963335990905762,
  0.0004024305671919137,
  0.007301412057131529,
  0.0010670725023373961,
  0

In [5]:
def BIN_config_DBPE():
    config = {}
    config['batch_size'] = [4,32,64]
    config['input_dim_drug'] = 23532
    config['input_dim_target'] = 16693
    config['max_drug_seq'] = 50 #64
    config['max_protein_seq'] = 545 #576
    config['emb_size'] = 384
    config['dropout_rate'] = [0.1,0.15,0.18]

    #DenseNet
    # config['scale_down_ratio'] = 0.25
    # config['growth_rate'] = 20
    # config['transition_rate'] = 0.5
    # config['num_dense_blocks'] = 4
    config['kernal_dense_size'] = 3

    # Encoder
    config['intermediate_size'] = 1536
    config['num_attention_heads'] = 12
    config['attention_probs_dropout_prob'] = [0.1,0.15,0.18]
    config['hidden_dropout_prob'] = [0.1,0.15,0.18]
    config['flat_dim'] = 78192 #106764
    return config

In [6]:
def main():
  batchsize = 64 # 16
  epochs =150 # 50
  lr1 = 1e-4
  confidence=0.55
  ratio1=15
  ratio2=31
  config = BIN_config_DBPE()
  model3 = BIN_Interaction_Flat3(**config)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_cuda else "cpu")

  dataFolder = '/content/drive/MyDrive/mini'
  df = pd.read_csv(dataFolder + '/train_new.csv')
  df1 = pd.read_csv(dataFolder + '/unlabel_acc_1.csv')
  df2 = pd.read_csv(dataFolder + '/unlabel_acc_2.csv')
  df3 = pd.read_csv(dataFolder + '/unlabel_acc_3.csv')
  df4 = pd.read_csv(dataFolder + '/unlabel_new.csv')
  df_train1=pd.concat([df,df1]).reset_index(drop=True)
  df_train2 = pd.concat([df,df2]).reset_index(drop=True)
  df_train3 = pd.concat([df,df3]).reset_index(drop=True)
  df_train4 = pd.concat([df,df4]).reset_index(drop=True)

  params = {'batch_size': batchsize, 'shuffle': True, 'drop_last': True}
  params2={'batch_size': 4, 'shuffle': True, 'drop_last': True}
  training_set = BIN_Data_Encoder(df_train1.index.values, df_train1.Label.values, df_train1)
  training_generator1 = DataLoader(training_set, **params)
  training_set = BIN_Data_Encoder(df_train2.index.values, df_train2.Label.values, df_train2)
  training_generator2 = DataLoader(training_set, **params)
  training_set = BIN_Data_Encoder(df_train3.index.values, df_train3.Label.values, df_train3)
  training_generator3 = DataLoader(training_set, **params)
  training_set = BIN_Data_Encoder(df_train4.index.values, df_train4.Label.values, df_train4)
  training_generator4 = DataLoader(training_set, **params)
  training_set = BIN_Data_Encoder(df.index.values, df.Label.values, df)
  training_generator = DataLoader(training_set, **params2)

  # print("-------phase1 0.85-------")
  # model_max, loss_history, loss_historyv = train(model3,training_generator1,epochs,batchsize,lr1)
  # torch.save(model_max, './new_test_acc2/model_max.pt') # 임시추가
  # loss1= pd.Series(loss_history)
  # loss1.to_csv('./new_test_acc2/loss1.csv',index=False)
  # loss1v= pd.Series(loss_historyv)
  # loss1v.to_csv('./new_test_acc2/loss1v.csv',index=False)
  # torch.cuda.empty_cache()

  # print("-------phase1 0.9-------")
  # model3 = BIN_Interaction_Flat3(**config)
  # model_max, loss_history, loss_historyv = train(model3,training_generator2,epochs,batchsize,lr1)
  # torch.save(model_max, './new_test_acc2/model_max2.pt') # 임시추가
  # loss1= pd.Series(loss_history)
  # loss1.to_csv('./new_test_acc2/loss2.csv',index=False)
  # loss1v= pd.Series(loss_historyv)
  # loss1v.to_csv('./new_test_acc2/loss2v.csv',index=False)
  # torch.cuda.empty_cache()

  print("-------phase1 0.95-------")
  model3 = BIN_Interaction_Flat3(**config)
  model_max, loss_history, loss_historyv = train(model3,training_generator3,epochs,batchsize,lr1)
  torch.save(model_max, './new_test_acc3/model_max30.pt') # 임시추가
  loss1= pd.Series(loss_history)
  loss1.to_csv('./new_test_acc3/loss30.csv',index=False)
  loss1v= pd.Series(loss_historyv)
  loss1v.to_csv('./new_test_acc3/loss30v.csv',index=False)
  torch.cuda.empty_cache()

  # print("-------phase1 1.0-------")
  # model3 = BIN_Interaction_Flat3(**config)
  # model_max, loss_history, loss_historyv = train(model3,training_generator4,epochs,batchsize,lr1)
  # torch.save(model_max, './new_test_acc/model_max4.pt') # 임시추가
  # loss1= pd.Series(loss_history)
  # loss1.to_csv('./new_test_acc/loss4.csv',index=False)
  # loss1v= pd.Series(loss_historyv)
  # loss1v.to_csv('./new_test_acc/loss4v.csv',index=False)
  # torch.cuda.empty_cache()

  # print("-------phase5 test--------")
  # model_max=torch.load('./new_test_acc2/model_max.pt')
  # model_max.batch_size=4
  # model3 = BIN_Interaction_Flat3(**config)
  # training_generator_with_unlabel=Cus_DataLoader(training_generator, df4, model_max, device, confidence, ratio1, 4)
  # torch.cuda.empty_cache()
  # model_comparison2, loss_history_comparison2, loss_history_comparison2v = train(model3, training_generator_with_unlabel, epochs*2, batchsize, lr1)
  # torch.save(model_comparison2, './new_test_acc2/model_comparison.pt') # 임시추가
  # loss_comparison2= pd.Series(loss_history_comparison2)
  # loss_comparison2.to_csv('./new_test_acc2/loss_comparison.csv',index=False)
  # loss_comparison2v= pd.Series(loss_history_comparison2v)
  # loss_comparison2v.to_csv('./new_test_acc2/loss_comparisonv.csv',index=False)
  # torch.cuda.empty_cache()


  print("-------training finished-------")

  return

In [7]:
s = time()
main()
e = time()
print((e - s)/60)

# from google.colab import runtime
# runtime.unassign()

-------phase1 0.95-------
--- Data Preparation ---
optimal threshold: 0.3653770089149475
AUROC:0.5141112412145572
AUPRC: 0.5219110367383657
Confusion Matrix : 
 [[   4 1331]
 [   1 1352]]
Recall :  0.9992609016999261
Precision :  0.5039135296310101
Accuracy :  0.5044642857142857
Sensitivity :  0.00299625468164794
Specificity :  0.9992609016999261
Initial Testing AUROC: 0.5141112412145572 , AUPRC: 0.5219110367383657 , F1: 0.33962264150943394 , Test loss: 0.695799667210806
--- Go for Training ---


training: 469it [02:01,  3.86it/s]


optimal threshold: 0.29982370138168335
AUROC:0.8302802072307949
AUPRC: 0.8697305674126257
Confusion Matrix : 
 [[412 207]
 [138 587]]
Recall :  0.8096551724137931
Precision :  0.739294710327456
Accuracy :  0.7433035714285714
Sensitivity :  0.6655896607431341
Specificity :  0.8096551724137931
--------1model_max updated-------
Validation at Epoch 1 , AUROC: 0.8302802072307949 , AUPRC: 0.8697305674126257 , F1: 0.7252573238321457 , Validation loss: 0.5030277087574914


training: 469it [01:50,  4.24it/s]


optimal threshold: 0.4965628981590271
AUROC:0.8605506224621837
AUPRC: 0.8929320149393358
Confusion Matrix : 
 [[448 166]
 [135 595]]
Recall :  0.815068493150685
Precision :  0.7818659658344284
Accuracy :  0.7760416666666666
Sensitivity :  0.7296416938110749
Specificity :  0.815068493150685
--------2model_max updated-------
Validation at Epoch 2 , AUROC: 0.8605506224621837 , AUPRC: 0.8929320149393358 , F1: 0.7970430107526881 , Validation loss: 0.45974120639619376


...

Training at Epoch 50 with loss 0.07163562452488108
optimal threshold: 0.3831748068332672
AUROC:0.8771365162211294
AUPRC: 0.8968239458560121
Confusion Matrix : 
 [[477 138]
 [130 599]]
Recall :  0.821673525377229
Precision :  0.8127544097693351
Accuracy :  0.8005952380952381
Sensitivity :  0.775609756097561
Specificity :  0.821673525377229
Validation at Epoch 50 , AUROC: 0.8771365162211294 , AUPRC: 0.8968239458560121 , F1: 0.8099516240497581 , Validation loss: 0.9886753275280907


training: 469it [01:50,  4.24it/s]


optimal threshold: 0.15110048651695251
AUROC:0.8850451219973352
AUPRC: 0.8977076168835477
Confusion Matrix : 
 [[472 155]
 [104 613]]
Recall :  0.8549511854951185
Precision :  0.7981770833333334
Accuracy :  0.8072916666666666
Sensitivity :  0.7527910685805422
Specificity :  0.8549511854951185
Validation at Epoch 51 , AUROC: 0.8850451219973352 , AUPRC: 0.8977076168835477 , F1: 0.8134642356241234 , Validation loss: 0.9904816633179074


training: 469it [01:50,  4.24it/s]


optimal threshold: 0.21694372594356537
AUROC:0.8775511113586987
AUPRC: 0.894237007012062
Confusion Matrix : 
 [[479 140]
 [121 604]]
Recall :  0.833103448275862
Precision :  0.8118279569892473
Accuracy :  0.8058035714285714
Sensitivity :  0.7738287560581584
Specificity :  0.833103448275862
Validation at Epoch 52 , AUROC: 0.8775511113586987 , AUPRC: 0.894237007012062 , F1: 0.8128941836019622 , Validation loss: 1.0217402549017043
--- Go for Testing ---
optimal threshold: 0.5286419987678528
AUROC:0.8871443954480401
AUPRC: 0.8951457226364878
Confusion Matrix : 
 [[1006  329]
 [ 200 1153]]
Recall :  0.852180339985218
Precision :  0.7780026990553306
Accuracy :  0.8031994047619048
Sensitivity :  0.753558052434457
Specificity :  0.852180339985218
Testing AUROC: 0.8871443954480401 , AUPRC: 0.8951457226364878 , F1: 0.8109243697478992 , Test loss: 0.5952260863213312
-------training finished-------
98.95853799978892


In [5]:
def BIN_config_DBPE():
    config = {}
    config['batch_size'] = [4,32,64]
    config['input_dim_drug'] = 23532
    config['input_dim_target'] = 16693
    config['max_drug_seq'] = 50 #64
    config['max_protein_seq'] = 545 #576
    config['emb_size'] = 384
    config['dropout_rate'] = [0.1,0.15,0.22]

    #DenseNet
    # config['scale_down_ratio'] = 0.25
    # config['growth_rate'] = 20
    # config['transition_rate'] = 0.5
    # config['num_dense_blocks'] = 4
    config['kernal_dense_size'] = 3

    # Encoder
    config['intermediate_size'] = 1536
    config['num_attention_heads'] = 12
    config['attention_probs_dropout_prob'] = [0.1,0.15,0.22]
    config['hidden_dropout_prob'] = [0.1,0.15,0.22]
    config['flat_dim'] = 78192 #106764
    return config

In [6]:
def main():
  batchsize = 64 # 16
  epochs =150 # 50
  lr1 = 1e-4
  confidence=0.55
  ratio1=15
  ratio2=31
  config = BIN_config_DBPE()
  model3 = BIN_Interaction_Flat3(**config)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_cuda else "cpu")

  dataFolder = '/content/drive/MyDrive/mini'
  df = pd.read_csv(dataFolder + '/train_new.csv')
  df1 = pd.read_csv(dataFolder + '/unlabel_acc_1.csv')
  df2 = pd.read_csv(dataFolder + '/unlabel_acc_2.csv')
  df3 = pd.read_csv(dataFolder + '/unlabel_acc_3.csv')
  df4 = pd.read_csv(dataFolder + '/unlabel_new.csv')
  df_train1=pd.concat([df,df1]).reset_index(drop=True)
  df_train2 = pd.concat([df,df2]).reset_index(drop=True)
  df_train3 = pd.concat([df,df3]).reset_index(drop=True)
  df_train4 = pd.concat([df,df4]).reset_index(drop=True)

  params = {'batch_size': batchsize, 'shuffle': True, 'drop_last': True}
  params2={'batch_size': 4, 'shuffle': True, 'drop_last': True}
  training_set = BIN_Data_Encoder(df_train1.index.values, df_train1.Label.values, df_train1)
  training_generator1 = DataLoader(training_set, **params)
  training_set = BIN_Data_Encoder(df_train2.index.values, df_train2.Label.values, df_train2)
  training_generator2 = DataLoader(training_set, **params)
  training_set = BIN_Data_Encoder(df_train3.index.values, df_train3.Label.values, df_train3)
  training_generator3 = DataLoader(training_set, **params)
  training_set = BIN_Data_Encoder(df_train4.index.values, df_train4.Label.values, df_train4)
  training_generator4 = DataLoader(training_set, **params)
  training_set = BIN_Data_Encoder(df.index.values, df.Label.values, df)
  training_generator = DataLoader(training_set, **params2)

  # print("-------phase1 0.85-------")
  # model_max, loss_history, loss_historyv = train(model3,training_generator1,epochs,batchsize,lr1)
  # torch.save(model_max, './new_test_acc2/model_max.pt') # 임시추가
  # loss1= pd.Series(loss_history)
  # loss1.to_csv('./new_test_acc2/loss1.csv',index=False)
  # loss1v= pd.Series(loss_historyv)
  # loss1v.to_csv('./new_test_acc2/loss1v.csv',index=False)
  # torch.cuda.empty_cache()

  # print("-------phase1 0.9-------")
  # model3 = BIN_Interaction_Flat3(**config)
  # model_max, loss_history, loss_historyv = train(model3,training_generator2,epochs,batchsize,lr1)
  # torch.save(model_max, './new_test_acc2/model_max2.pt') # 임시추가
  # loss1= pd.Series(loss_history)
  # loss1.to_csv('./new_test_acc2/loss2.csv',index=False)
  # loss1v= pd.Series(loss_historyv)
  # loss1v.to_csv('./new_test_acc2/loss2v.csv',index=False)
  # torch.cuda.empty_cache()

  print("-------phase1 0.95-------")
  model3 = BIN_Interaction_Flat3(**config)
  model_max, loss_history, loss_historyv = train(model3,training_generator3,epochs,batchsize,lr1)
  torch.save(model_max, './new_test_acc3/model_max31.pt') # 임시추가
  loss1= pd.Series(loss_history)
  loss1.to_csv('./new_test_acc3/loss31.csv',index=False)
  loss1v= pd.Series(loss_historyv)
  loss1v.to_csv('./new_test_acc3/loss31v.csv',index=False)
  torch.cuda.empty_cache()

  # print("-------phase1 1.0-------")
  # model3 = BIN_Interaction_Flat3(**config)
  # model_max, loss_history, loss_historyv = train(model3,training_generator4,epochs,batchsize,lr1)
  # torch.save(model_max, './new_test_acc3/model_max4.pt') # 임시추가
  # loss1= pd.Series(loss_history)
  # loss1.to_csv('./new_test_acc3/loss4.csv',index=False)
  # loss1v= pd.Series(loss_historyv)
  # loss1v.to_csv('./new_test_acc3/loss4v.csv',index=False)
  # torch.cuda.empty_cache()

  # print("-------phase5 test--------")
  # model_max=torch.load('./new_test_acc2/model_max.pt')
  # model_max.batch_size=4
  # model3 = BIN_Interaction_Flat3(**config)
  # training_generator_with_unlabel=Cus_DataLoader(training_generator, df4, model_max, device, confidence, ratio1, 4)
  # torch.cuda.empty_cache()
  # model_comparison2, loss_history_comparison2, loss_history_comparison2v = train(model3, training_generator_with_unlabel, epochs*2, batchsize, lr1)
  # torch.save(model_comparison2, './new_test_acc2/model_comparison.pt') # 임시추가
  # loss_comparison2= pd.Series(loss_history_comparison2)
  # loss_comparison2.to_csv('./new_test_acc2/loss_comparison.csv',index=False)
  # loss_comparison2v= pd.Series(loss_history_comparison2v)
  # loss_comparison2v.to_csv('./new_test_acc2/loss_comparisonv.csv',index=False)
  # torch.cuda.empty_cache()


  print("-------training finished-------")

  return

In [7]:
s = time()
main()
e = time()
print((e - s)/60)

from google.colab import runtime
runtime.unassign()

-------phase1 0.95-------
--- Data Preparation ---
optimal threshold: 0.33949175477027893
AUROC:0.5139645288179133
AUPRC: 0.5239884416900522
Confusion Matrix : 
 [[   0 1335]
 [   0 1353]]
Recall :  1.0
Precision :  0.5033482142857143
Accuracy :  0.5033482142857143
Sensitivity :  0.0
Specificity :  1.0
Initial Testing AUROC: 0.5139645288179133 , AUPRC: 0.5239884416900522 , F1: 0.3458870168483647 , Test loss: 0.696020367599669
--- Go for Training ---


training: 469it [02:00,  3.88it/s]


optimal threshold: 0.9911670088768005
AUROC:0.6192067294301153
AUPRC: 0.6540632851012167
Confusion Matrix : 
 [[ 57 562]
 [ 25 700]]
Recall :  0.9655172413793104
Precision :  0.554675118858954
Accuracy :  0.5632440476190477
Sensitivity :  0.09208400646203554
Specificity :  0.9655172413793104
--------1model_max updated-------
Validation at Epoch 1 , AUROC: 0.6192067294301153 , AUPRC: 0.6540632851012167 , F1: 0.7008216529724505 , Validation loss: 2.7297780967894054


training: 469it [01:50,  4.26it/s]


optimal threshold: 0.9789936542510986
AUROC:0.7786410691178438
AUPRC: 0.8233706439884614
Confusion Matrix : 
 [[426 188]
 [199 531]]
Recall :  0.7273972602739726
Precision :  0.7385257301808067
Accuracy :  0.7120535714285714
Sensitivity :  0.6938110749185668
Specificity :  0.7273972602739726
--------2model_max updated-------
Validation at Epoch 2 , AUROC: 0.7786410691178438 , AUPRC: 0.8233706439884614 , F1: 0.7202072538860104 , Validation loss: 1.3594875080244881


...

optimal threshold: 0.9480088353157043
AUROC:0.9152730892216154
AUPRC: 0.9308926346558145
Confusion Matrix : 
 [[543  79]
 [125 597]]
Recall :  0.8268698060941828
Precision :  0.8831360946745562
Accuracy :  0.8482142857142857
Sensitivity :  0.8729903536977492
Specificity :  0.8268698060941828
Validation at Epoch 77 , AUROC: 0.9152730892216154 , AUPRC: 0.9308926346558145 , F1: 0.8524137931034482 , Validation loss: 1.0543916083517528


training: 469it [01:50,  4.26it/s]


optimal threshold: 0.692349910736084
AUROC:0.9114467713320318
AUPRC: 0.9280231754769382
Confusion Matrix : 
 [[514 104]
 [115 611]]
Recall :  0.8415977961432507
Precision :  0.8545454545454545
Accuracy :  0.8370535714285714
Sensitivity :  0.8317152103559871
Specificity :  0.8415977961432507
Validation at Epoch 78 , AUROC: 0.9114467713320318 , AUPRC: 0.9280231754769382 , F1: 0.8424657534246576 , Validation loss: 1.1440085201036363


training: 469it [01:50,  4.26it/s]


optimal threshold: 0.9686318039894104
AUROC:0.9037707867570431
AUPRC: 0.9232156435692975
Confusion Matrix : 
 [[525  97]
 [122 600]]
Recall :  0.8310249307479224
Precision :  0.860832137733142
Accuracy :  0.8370535714285714
Sensitivity :  0.8440514469453376
Specificity :  0.8310249307479224
Validation at Epoch 79 , AUROC: 0.9037707867570431 , AUPRC: 0.9232156435692975 , F1: 0.8324468085106383 , Validation loss: 1.0338308328673953
--- Go for Testing ---
optimal threshold: 0.8965975642204285
AUROC:0.907444172306275
AUPRC: 0.9180556532383667
Confusion Matrix : 
 [[1095  241]
 [ 204 1148]]
Recall :  0.849112426035503
Precision :  0.8264938804895609
Accuracy :  0.8344494047619048
Sensitivity :  0.8196107784431138
Specificity :  0.849112426035503
Testing AUROC: 0.907444172306275 , AUPRC: 0.9180556532383667 , F1: 0.8355146798726565 , Test loss: 1.047334231081463
-------training finished-------
149.2926996787389
